#Importing files from Google Drive

In [1]:
!pip install PyDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"16_ZNThSbAQyY1Hw6CXBm6hteZ0eeLJpI"})   # replace the id with id of file you want to access
downloaded.GetContentFile('pytorch_model.bin')        # replace the file name with your file
downloaded = drive.CreateFile({'id':"1ill09R5sdg7GzBCfKfGty8eW5F9YZ-m1"})   # replace the id with id of file you want to access
downloaded.GetContentFile('config.json')

#Code started

In [0]:
!pip install transformers==2.4.1 -q

In [0]:
import torch
from transformers import *

model_name = 'roberta-large' #uncased should have do_lower_case=True
model = AutoModelForSequenceClassification.from_pretrained('./')
tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=False)

In [43]:
model.classifier

RobertaClassificationHead(
  (dense): Linear(in_features=1024, out_features=1024, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=1024, out_features=2, bias=True)
)

In [0]:
models_names = [ 
'bert-base-uncased',
'bert-large-uncased',
'bert-base-cased',
'bert-large-cased',
'bert-base-multilingual-uncased',
'bert-base-multilingual-cased',
'bert-large-uncased-whole-word-masking',
'bert-large-cased-whole-word-masking',
'bert-large-uncased-whole-word-masking-finetuned-squad',
'bert-large-cased-whole-word-masking-finetuned-squad',
'bert-base-cased-finetuned-mrpc',
'albert-base-v1',
'albert-large-v1',
'albert-xlarge-v1',
'albert-xxlarge-v1',
'albert-base-v2',
'albert-large-v2',
'albert-xlarge-v2',
'albert-xxlarge-v2',
'xlm-mlm-en-2048',
'xlm-mlm-ende-1024',
'xlm-mlm-enfr-1024',
'xlm-mlm-enro-1024',
'xlm-mlm-xnli15-1024',
'xlm-mlm-tlm-xnli15-1024',
'xlm-clm-enfr-1024',
'xlm-clm-ende-1024',
'xlm-mlm-17-1280',
'xlm-mlm-100-1280',
'xlm-roberta-base',
'xlm-roberta-large',
'roberta-base',
'roberta-large',
'roberta-large-mnli',
'roberta-base-openai-detector',
'roberta-large-openai-detector',
'xlnet-base-cased',
'xlnet-large-cased',
]

# RoBERTa Fine-Tuning Tutorial with PyTorch

# 1. Setup

## 1.1. Using Colab GPU for Training


In [45]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [46]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 1.2. Installing the Hugging Face Library


# 2. Loading SemEval Dataset


## 2.1. Download & Extract

We'll use the `wget` package to download the dataset to the Colab instance's file system. 

In [47]:
!pip install wget

The dataset is hosted on GitHub in this repo: https://nyu-mll.github.io/CoLA/

In [48]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_data_all.csv
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_answers_all.csv

#trial
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Trial%20Data/taskA_trial_data.csv
!wget -q https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Trial%20Data/taskA_trial_answer.csv
#dev
!wget https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Dev%20Data/subtaskA_gold_answers.csv
!wget https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Dev%20Data/subtaskA_dev_data.csv

#test
!wget https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Test%20Data/subtaskA_test_data.csv

--2020-02-21 18:05:32--  https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Dev%20Data/subtaskA_gold_answers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6443 (6.3K) [text/plain]
Saving to: ‘subtaskA_gold_answers.csv.1’

subtaskA_gold_answe 100%[===================>]   6.29K  --.-KB/s    in 0s      

2020-02-21 18:05:33 (156 MB/s) - ‘subtaskA_gold_answers.csv.1’ saved [6443/6443]

--2020-02-21 18:05:33--  https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Dev%20Data/subtaskA_dev_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubuserconten

## 2.2. Parse


We can see from the file names that both `tokenized` and `raw` versions of the data are available. 

We can't use the pre-tokenized version because, in order to apply the pre-trained BERT, we *must* use the tokenizer provided by the model. This is because (1) the model has a specific, fixed vocabulary and (2) the BERT tokenizer has a particular way of handling out-of-vocabulary words.

We'll use pandas to parse the "in-domain" training set and look at a few of its properties and data points.

##training set

In [49]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe.
df_data = pd.read_csv('subtaskA_data_all.csv', error_bad_lines=False)
df_answers = pd.read_csv('subtaskA_answers_all.csv', error_bad_lines=False, header=None, names=['id', 'label'])

df = df_data.set_index('id').join(df_answers.set_index('id'))
df.reset_index(drop=True, inplace=True)

for i in range(len(df)):
    if df['sent0'][i][-1] != '.' or df['sent1'][i][-1] !='.' :
      df['sent0'][i] = df['sent0'][i] +'.'
      df['sent1'][i] = df['sent1'][i] +'.'
      
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.head(10)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Number of training sentences: 10,000



,sent0,sent1,label
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today.,"Jeff ran 100,000 miles today.",1
3,A mosquito stings me.,I sting a mosquito.,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie.,Mark ate a big sweet cherry pie.,0
9996,Gloria wears a cat on her head.,Gloria wears a hat on her head.,0
9997,Harry went to the barbershop to have his hair ...,Harry went to the barbershop to have his glass...,1
9998,Reilly is sleeping on the couch.,Reilly is sleeping on the window.,1


##trial set

In [50]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe.
df_data = pd.read_csv('taskA_trial_data.csv', error_bad_lines=False)
df_answers = pd.read_csv('taskA_trial_answer.csv', error_bad_lines=False, header=None, names=['id', 'label'])

df_trial = df_data.set_index('id').join(df_answers.set_index('id'))
df_trial.reset_index(drop=True, inplace=True)

for i in range(len(df_trial)):
    if df_trial['sent0'][i][-1] != '.' or df_trial['sent1'][i][-1] !='.' :
      df_trial['sent0'][i] = df_trial['sent0'][i] +'.'
      df_trial['sent1'][i] = df_trial['sent1'][i] +'.'
      
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_trial.shape[0]))

# Display 10 random rows from the data.
df_trial.head(10)
df_trial

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Number of training sentences: 2,021



,sent0,sent1,label
0,he put an elephant into the fridge.,he put a turkey into the fridge.,0
1,my sister eats an apple after breakfast every ...,my sister eats a stone after breakfast every day.,1
2,money can be used for buying cars.,money can be used for buying stars.,1
3,New York is located in the northeastern part o...,USA is located in the northeastern part of New...,1
4,a man can better see stars and the moon in day...,a man can hardly see stars and the moon in day...,0
...,...,...,...
2016,she cleans her teeth with toothpaste.,she cleans her teeth with shampoo.,1
2017,you can see a dinosaur in the museum.,you can see a dinosaur in the zoo.,1
2018,people have to hold onto their hats because of...,people have to hold onto their shoes because o...,1
2019,Rebecca wakes up because she takes sleeping pi...,Rebecca wakes up because she sets an alarm.,0


##training set + trial set

In [51]:
frames = [df, df_trial]
df = pd.concat(frames, ignore_index=True)
df

,sent0,sent1,label
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today.,"Jeff ran 100,000 miles today.",1
3,A mosquito stings me.,I sting a mosquito.,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
12016,she cleans her teeth with toothpaste.,she cleans her teeth with shampoo.,1
12017,you can see a dinosaur in the museum.,you can see a dinosaur in the zoo.,1
12018,people have to hold onto their hats because of...,people have to hold onto their shoes because o...,1
12019,Rebecca wakes up because she takes sleeping pi...,Rebecca wakes up because she sets an alarm.,0


##dev set

In [52]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe.
df_data = pd.read_csv('subtaskA_dev_data.csv', error_bad_lines=False)
df_answers = pd.read_csv('subtaskA_gold_answers.csv', error_bad_lines=False, header=None, names=['id', 'label'])

df_dev = df_data.set_index('id').join(df_answers.set_index('id'))
df_dev.reset_index(drop=True, inplace=True)

for i in range(len(df_dev)):
    if df_dev['sent0'][i][-1] != '.' or df_dev['sent1'][i][-1] !='.' :
      df_dev['sent0'][i] = df_dev['sent0'][i] +'.'
      df_dev['sent1'][i] = df_dev['sent1'][i] +'.'
      
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_dev.shape[0]))

# Display 10 random rows from the data.
df_dev.head(10)

Number of training sentences: 997



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sent0,sent1,label
0,"Summer in North America is great for skiing, ...","Summer in North America is great for swimming,...",0
1,You can use detergent to dye your hair.,You can use bleach to dye your hair.,0
2,passing your driving license exams requires st...,passing your university exams requires studyin...,0
3,The hangers bought the closet.,The closet got hangers.,0
4,coffee takes sleep.,coffee depresses people.,1
5,The cat used the litter box.,The cat used the toilet.,1
6,He wrote an exam in pen.,He wrote an exam in knife.,1
7,she gave birth to a baby.,he gave birth to a baby.,1
8,Chicken are found on a farm.,Chicken can swim in water.,1
9,A stick wants to have sex.,A person wants to have sex.,0


##test set

In [53]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe.
df_test = pd.read_csv('subtaskA_test_data.csv', error_bad_lines=False)
df_test

,id,sent0,sent1
0,1175,He loves to stroll at the park with his bed,He loves to stroll at the park with his dog.
1,452,The inverter was able to power the continent.,The inverter was able to power the house
2,275,The chef put extra lemons on the pizza.,The chef put extra mushrooms on the pizza.
3,869,sugar is used to make coffee sour,sugar is used to make coffee sweet
4,50,There are beautiful flowers here and there in ...,There are beautiful planes here and there in t...
...,...,...,...
995,1114,"If it had rained, you would got wet.","If it is a sunny day, you would got wet."
996,8,ice hockey is a sport,ice hockey is a financial institution
997,1945,He put water without a container in the freeze...,He put a watermelon in the freezer for 24 hours
998,1053,The desert has sand that you can drink.,"The desert is very dry, so bring water when yo..."


#Let's extract the sentences and labels of our training set as numpy ndarrays.

In [54]:
# Get the lists of sentences and their labels.
sentence1 = df.sent1.values
sentence0 = df.sent0.values
labels = df.label.values



#test data
#sentence1 = df_test.sent1.values
#sentence0 = df_test.sent0.values
#test_labels = df_test.label.values


#dev
sentence1 = df_dev.sent1.values
sentence0 = df_dev.sent0.values
dev_labels = df_dev.label.values

dev_labels

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,

# 3. Tokenization & Input Formatting

In this section, we'll transform our dataset into the format that BERT can be trained on.

## 3.2. Required Formatting

## 3.2. Sentences to IDs using tokenizer.encode()

In [0]:
MAX_LEN = 64

##encode_plus for training + trial data

In [56]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
input_type_ids = []
attention_masks = []

# For every sentence...
for i in range(len(df)):
    tokenizer.padding_side = 'right'
    encoded_sent = tokenizer.encode_plus(
                        df['sent0'][i],            # Sentence to encode.
                        df['sent1'][i],
                        add_special_tokens = True, 
                        max_length = MAX_LEN,
                        pad_to_max_length = True
                        )

    #Add the encoded sentence to the list.
    input_ids.append(encoded_sent['input_ids'])
    input_type_ids.append(encoded_sent['token_type_ids'])
    attention_masks.append(encoded_sent['attention_mask'])

#Print last sentences of df, now as a list of IDs.
print('original: ' , df['sent0'][0], df['sent1'][0])
print('Token IDs:', input_ids[10])
print('Type IDs:', input_type_ids[10])
print('MASK IDs:', attention_masks[10])

#Original:  I have a desk on my lamp.
#Tokenized:  ['▁', 'I', '▁have', '▁a', '▁desk', '▁on', '▁my', '▁lamp', '.']
#Token IDs:  [13, 1, 57, 21, 2911, 27, 51, 6792, 9]
tokenizer.cls_token_id, tokenizer.sep_token_id

original:  He poured orange juice on his cereal. He poured milk on his cereal.
Token IDs: [0, 717, 6149, 29, 3529, 449, 354, 15, 9274, 4, 2, 2, 32180, 465, 7689, 15, 9274, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
MASK IDs: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(0, 2)

##encode_plus for test data

In [57]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
validation_inputs = []
validation_type_ids = []
validation_masks = []

# For every sentence...
for i in range(len(df_test)):
    tokenizer.padding_side = 'right'
    encoded_sent = tokenizer.encode_plus(
                        df_test['sent0'][i],            # Sentence to encode.
                        df_test['sent1'][i],
                        add_special_tokens = True, 
                        max_length = MAX_LEN,
                        pad_to_max_length = True
                        )

    #Add the encoded sentence to the list.
    validation_inputs.append(encoded_sent['input_ids'])
    validation_type_ids.append(encoded_sent['token_type_ids'])
    validation_masks.append(encoded_sent['attention_mask'])

#Print last sentences of df, now as a list of IDs.
print('original: ' , df_test['sent0'][0], df_test['sent1'][0])
print('Token IDs:', validation_inputs[10])
print('Type IDs:', validation_type_ids[10])
print('MASK IDs:', validation_masks[10])

#Original:  I have a desk on my lamp.
#Tokenized:  ['▁', 'I', '▁have', '▁a', '▁desk', '▁on', '▁my', '▁lamp', '.']
#Token IDs:  [13, 1, 57, 21, 2911, 27, 51, 6792, 9]
tokenizer.cls_token_id, tokenizer.sep_token_id

original:  He loves to stroll at the park with his bed He loves to stroll at the park with his dog.
Token IDs: [0, 133, 2931, 1326, 66, 69, 2, 2, 2515, 1326, 66, 5, 2931, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
MASK IDs: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(0, 2)

## 3.5. Training & Validation Split


##The following code split 10 % of training set as validation **set**

In [0]:
# Use train_test_split to split our data into train and validation sets for training
from sklearn.model_selection import train_test_split
import random

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels, train_type_ids, validation_type_ids, train_masks, validation_masks = (
    train_test_split(input_ids, labels, input_type_ids, attention_masks, random_state=2018, test_size=0.1))


##The following set has all training data for training set and use trial data for validation set

In [0]:
# Use train_test_split to split our data into train and validation sets for training
from sklearn.model_selection import train_test_split
import random

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

train_inputs, train_labels, train_type_ids, train_masks = input_ids, labels , input_type_ids , attention_masks


validation_inputs,  validation_type_ids, validation_masks = validation_inputs, validation_type_ids, validation_masks


## 3.6. Converting to PyTorch Data Types

In [0]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.

train_type_ids = torch.tensor(train_type_ids) 
validation_type_ids = torch.tensor(validation_type_ids)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
#validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels, train_type_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_type_ids)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# 4. Train Our Classification Model

## 4.2. Optimizer & Learning Rate Scheduler

In [0]:
model.cuda();
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [62]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 20

# Total number of training steps is number of batches * number of epochs.
print('len(train_dataloader)', len(train_dataloader))
total_steps = len(train_dataloader) * epochs
print('total_steps', total_steps)
warmup_steps = int(0.06 * total_steps)
print('warmup_steps', warmup_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, # Default value in run_glue.py
                                            num_training_steps = total_steps)

len(train_dataloader) 752
total_steps 15040
warmup_steps 902


## 4.3. Training Loop

###Define a helper function for calculating accuracy.

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


We're ready to kick off the training!

In [65]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}    LR . {:.2E}'.format(step, len(train_dataloader), elapsed, scheduler.get_lr()[0]))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_type_ids = batch[3].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=b_type_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    pred = []

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_type_ids = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=b_type_ids, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        pred.append(list(np.argmax(logits, axis=1).flatten()))
        
        # Accumulate the total accuracy.
        #eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    #print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


  Batch    40  of    752.    Elapsed: 0:00:32    LR . 4.43E-07
  Batch    80  of    752.    Elapsed: 0:01:03    LR . 8.87E-07
  Batch   120  of    752.    Elapsed: 0:01:34    LR . 1.33E-06
  Batch   160  of    752.    Elapsed: 0:02:06    LR . 1.77E-06
  Batch   200  of    752.    Elapsed: 0:02:37    LR . 2.22E-06
  Batch   240  of    752.    Elapsed: 0:03:08    LR . 2.66E-06
  Batch   280  of    752.    Elapsed: 0:03:40    LR . 3.10E-06
  Batch   320  of    752.    Elapsed: 0:04:11    LR . 3.55E-06
  Batch   360  of    752.    Elapsed: 0:04:42    LR . 3.99E-06
  Batch   400  of    752.    Elapsed: 0:05:14    LR . 4.43E-06
  Batch   440  of    752.    Elapsed: 0:05:45    LR . 4.88E-06
  Batch   480  of    752.    Elapsed: 0:06:16    LR . 5.32E-06
  Batch   520  of    752.    Elapsed: 0:06:47    LR . 5.76E-06
  Batch   560  of    752.    Elapsed: 0:07:19    LR . 6.21E-06
  Batch   600  of    752.    Elapsed: 0:07:50    LR . 6.65E-06
  Batch   640  of    752.    Elapsed: 0:08:21    LR . 7

In [66]:
pred

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1],
 [0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0,

In [67]:
import itertools

preds = list(itertools.chain(*pred))
preds

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,


In [68]:
df_test['prediction'] = 0
df_test

,id,sent0,sent1,prediction
0,1175,He loves to stroll at the park with his bed,He loves to stroll at the park with his dog.,0
1,452,The inverter was able to power the continent.,The inverter was able to power the house,0
2,275,The chef put extra lemons on the pizza.,The chef put extra mushrooms on the pizza.,0
3,869,sugar is used to make coffee sour,sugar is used to make coffee sweet,0
4,50,There are beautiful flowers here and there in ...,There are beautiful planes here and there in t...,0
...,...,...,...,...
995,1114,"If it had rained, you would got wet.","If it is a sunny day, you would got wet.",0
996,8,ice hockey is a sport,ice hockey is a financial institution,0
997,1945,He put water without a container in the freeze...,He put a watermelon in the freezer for 24 hours,0
998,1053,The desert has sand that you can drink.,"The desert is very dry, so bring water when yo...",0


In [69]:
for i in range(len(df_test)):
  df_test['prediction'][i] = preds [i]

df_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,sent0,sent1,prediction
0,1175,He loves to stroll at the park with his bed,He loves to stroll at the park with his dog.,0
1,452,The inverter was able to power the continent.,The inverter was able to power the house,0
2,275,The chef put extra lemons on the pizza.,The chef put extra mushrooms on the pizza.,0
3,869,sugar is used to make coffee sour,sugar is used to make coffee sweet,0
4,50,There are beautiful flowers here and there in ...,There are beautiful planes here and there in t...,1
...,...,...,...,...
995,1114,"If it had rained, you would got wet.","If it is a sunny day, you would got wet.",1
996,8,ice hockey is a sport,ice hockey is a financial institution,1
997,1945,He put water without a container in the freeze...,He put a watermelon in the freezer for 24 hours,0
998,1053,The desert has sand that you can drink.,"The desert is very dry, so bring water when yo...",0


In [70]:
df_test = df_test[['id' , 'prediction']]
df_test.to_csv('df_test.csv' , index=False, header=False)
df_test

,id,prediction
0,1175,0
1,452,0
2,275,0
3,869,0
4,50,1
...,...,...
995,1114,1
996,8,1
997,1945,0
998,1053,0


In [0]:
from google.colab import files
files.download("df_test.csv")

##Let's take a look at our training loss over all batches:

In [0]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o')

# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

# 5. Performance On Test Set

Now we'll load the holdout dataset and prepare inputs just as we did with the training set. Then we'll evaluate predictions using [Matthew's correlation coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html) because this is the metric used by the wider NLP community to evaluate performance on CoLA. With this metric, +1 is the best score, and -1 is the worst score. This way, we can see how well we perform against the state of the art models for this specific task.

### 5.1. Data Preparation



We'll need to apply all of the same steps that we did for the training data to prepare our test data set.

In [0]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df_data = pd.read_csv('taskA_trial_data.csv', error_bad_lines=False)
df_answers = pd.read_csv('taskA_trial_answer.csv', error_bad_lines=False, header=None, names=['id', 'label'])
df = df_data.set_index('id').join(df_answers.set_index('id'))

df_appended = np.concatenate([df['sent0'] , df['sent1']])
df = pd.DataFrame(df_appended, columns=['sentences'], )
df.index.name = 'id'

df_fliped = df_answers.replace({0:1, 1:0})
df_Concat_answers = np.concatenate([df_fliped['label'] , df_answers['label']])
df_Concat_answers = pd.DataFrame(df_Concat_answers , columns=['label'])
df_Concat_answers.index.name = 'id'
df = df_Concat_answers.merge(df , on='id')

for i in range(len(df)):
    if df['sentences'][i][-1] != '.':
      df['sentences'][i] = df['sentences'][i] +'.'

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = df.sentences.values
labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## 5.2. Evaluate on Test Set



With the test set prepared, we can apply our fine-tuned model to generate predictions on the test set.

In [0]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Accuracy on the CoLA benchmark is measured using the "[Matthews correlation coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html)" (MCC).

We use MCC here because the classes are imbalanced:


In [0]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [0]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

The final score will be based on the entire test set, but let's take a look at the scores on the individual batches to get a sense of the variability in the metric between batches. 

Each batch has 32 sentences in it, except the last batch which has only (516 % 32) = 4 test sentences in it.


In [0]:
matthews_set

In [0]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

Cool! In about half an hour and without doing any hyperparameter tuning (adjusting the learning rate, epochs, batch size, ADAM properties, etc.) we are able to get a good score. I should also mention we didn't train on the entire training dataset, but set aside a portion of it as our validation set for legibililty of code.

The library documents the expected accuracy for this benchmark [here](https://huggingface.co/transformers/examples.html#glue).

You can also look at the official leaderboard [here](https://gluebenchmark.com/leaderboard/submission/zlssuBTm5XRs0aSKbFYGVIVdvbj1/-LhijX9VVmvJcvzKymxy). 

Note that (due to the small dataset size?) the accuracy can vary significantly with different random seeds.


# Conclusion

This post demonstrates that with a pre-trained BERT model you can quickly and effectively create a high quality model with minimal effort and training time using the pytorch interface, regardless of the specific NLP task you are interested in.

# Appendix


## A1. Saving & Loading Fine-Tuned Model

This first cell (taken from `run_glue.py` [here](https://github.com/huggingface/transformers/blob/35ff345fc9df9e777b27903f11fa213e4052595b/examples/run_glue.py#L495)) writes the model and tokenizer out to disk.

In [0]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Let's check out the file sizes, out of curiosity.

In [0]:
!ls -l --block-size=K ./model_save/

The largest file is the model weights, at around 418 megabytes.

In [0]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

To save your model across Colab Notebook sessions, download it to your local machine, or ideally copy it to your Google Drive.

In [0]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
    drive.mount('/content/drive')

In [0]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"

The following functions will load the model back from disk.

In [0]:
# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

## A.2. Weight Decay



The huggingface example includes the following code block for enabling weight decay, but the default decay rate is "0.0", so I moved this to the appendix.

This block essentially tells the optimizer to not apply weight decay to the bias terms (e.g., $ b $ in the equation $ y = Wx + b $ ). Weight decay is a form of regularization--after calculating the gradients, we multiply them by, e.g., 0.99.

In [0]:
# This code is taken from:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102

# Don't apply weight decay to any parameters whose names include these tokens.
# (Here, the BERT doesn't have `gamma` or `beta` parameters, only `bias` terms)
no_decay = ['bias', 'LayerNorm.weight']

# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# Note - `optimizer_grouped_parameters` only includes the parameter values, not 
# the names.